# Import Libraries and Construct Data

In [1]:
from __future__ import division
import pickle
import random
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

In [2]:
raw_data = dict()
raw_data['train_data'] = np.load("training_arr.npy")
raw_data['test_data_pos'] = np.load("biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load("biased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

# Define & Load Model

In [3]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix="./yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2023-12-26 23:02:13.464555: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-12-26 23:02:13.514745: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7faf539cbdb0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-26 23:02:13.514766: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [4]:
cml_model.load("cml-yahoo")

INFO:tensorflow:Restoring parameters from cml-yahoo


# Generate Raw Results

In [5]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = "cml-yahoo-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = "cml-yahoo-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 15400/15400 [06:04<00:00, 42.30it/s]


{'AUC': [0.47289830508474573,
  0.4968537414965987,
  0.5127986348122867,
  0.4837166666666667,
  0.45263422818791943,
  0.5002188552188552,
  0.5026288659793815,
  0.4915979381443299,
  0.5277999999999999,
  0.46397306397306404,
  0.4812080536912752,
  0.49238255033557043,
  0.45790540540540536,
  0.5086409395973155,
  0.49210884353741496,
  0.4621525423728813,
  0.5065500000000001,
  0.4554391891891892,
  0.5166053511705686,
  0.4894387755102041,
  0.5121333333333333,
  0.4836,
  0.46962711864406775,
  0.4579026845637583,
  0.5080102040816327,
  0.5297979797979798,
  0.4752040816326531,
  0.4916589861751152,
  0.5000501672240802,
  0.5147610921501706,
  0.5046166666666666,
  0.4923578595317726,
  0.5227702702702703,
  0.5221043771043771,
  0.4863545150501672,
  0.4933163265306123,
  0.5394648829431439,
  0.4697166666666666,
  0.48646666666666666,
  0.5054362416107383,
  0.5266166666666667,
  0.51695,
  0.5286195286195287,
  0.49135,
  0.50815,
  0.4768288590604027,
  0.50141666666666

# Evaluation

In [6]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < 1:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [8]:
eq("cml-yahoo-test-pos-biased_evaluate_partial.pickle", "cml-yahoo-test-neg-biased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

{'auc': 0.6582512516831484, 'recall': 0.01215594999877075}